In [1]:
import numpy as np
from typeguard import typechecked
from decimal import Decimal
from IPython.display import display, Latex
import numbers
from uncertainties import ufloat
import uncertainties
from pint import UnitRegistry, DimensionalityError
ureg = UnitRegistry(non_int_type=Decimal)
ureg.default_format = "~P"
Q_=ureg.Quantity

esc = lambda s: s.replace('"\"','"\\"')

from interval import interval, inf, imath

In [2]:
class Limits:
    def __init__(self, limits):
        lower, upper = limits
        try:
            assert lower.dimensionality == upper.dimensionality
            if not isinstance(lower.m, Decimal):
                assert lower.m.std_dev == 0
            if not isinstance(upper.m, Decimal):
                assert upper.m.std_dev == 0
        except DimensionalityError as exc:
            raise exc
        lower.ito_base_units()
        upper.ito_base_units()
        self.value = interval([lower.m, upper.m])
        self.units = lower.units
    
    def __mul__(self,other):
        if isinstance(other, numbers.Number):
            return self.value*other
        elif isinstance(other, Limits):
            return self.value*other.value

    def __rmul__(self,other):
        return self.__mul__(other)
    
    def __truediv__(self,other):
        if isinstance(other, numbers.Number):
            return self.value/other
        elif isinstance(other, Limits):
            return self.value/other.value
        
    def __repr__(self):
        return self.value.__str__() + self.units.__str__()
l = Limits([Q_('2 kg'),Q_('6 kg')])

In [3]:
I = interval([0,1])

In [23]:
class BasicQuantity:
    units = '1'
    class_units = Q_(f"{units}")
    def __init__(self, name="", **kwargs):
        c1 = set(kwargs.keys()) == {'magnitude','units'}
        c2 = set(kwargs.keys()) == {'quantity'}
        c3 = set(kwargs.keys()) == {'limits'}
        self.name = name
        
        assert c1 ^ c2 ^ c3
        if c1:
            magnitude = kwargs['magnitude']
            units = kwargs['units']
            if isinstance(magnitude,numbers.Number):
                self.magnitude = ufloat(magnitude,0)
            else:
                self.magnitude = magnitude
                self.units = units
            self.value = Q_(self.magnitude, self.units)
        if c2:
            quantity = kwargs['quantity']
            self.value = quantity          
        if c3:
            limits = kwargs['limits']
            self.value = limits 
            
#         try:
#             if c1 or c2:
#                 self.value.to(self.class_units)
#         except DimensionalityError as exc:
#             raise exc

    def __add__(self,other):
        try:
            return self.value + other.value
        except DimensionalityError as exc:
            raise exc

    def __sub__(self, other):
        try:
            return self.value - other.value
        except DimensionalityError as exc:
            raise exc
            
    def __mul__(self,other):
        if isinstance(other, numbers.Number):
            return self.value*other
        elif isinstance(other, BasicQuantity):
            return self.value*other.value

    def __rmul__(self,other):
        return self.__mul__(other)
    
    def __pow__(self,n):
        return self.value**n
        
    def __truediv__(self,other):
        if isinstance(other, numbers.Number):
            return self.value/other
        elif isinstance(other, BasicQuantity):
            return self.value/other.value
        
    def __str__(self):
        if isinstance(self.value, interval):
            h = self.value.__str__()
            u = self.units
        else:
            h = self.value.m
            u = self.value.units
        s = f"{self.name} = {h} \\, \\rm{{[{u}]}}, \\, \\rm{{({self.__class__.__name__})}}"
        s = s.replace('+/-',r'\pm')
        return s
    
    def __repr__(self):
        s = f"$${self.__str__()}$$"
        display(Latex(s))
        return ""

def quantity_maker(klass, units, expression=lambda x:Q_('0')):
    return type(klass,(BasicQuantity,),{"class_units":Q_(units),'units':units ,'expression':expression})
    


In [24]:
# custom physical quanti    
Mass = quantity_maker('Mass','kg')
Time = quantity_maker('Mass','sec')
Length = quantity_maker('Length','m')
Area = quantity_maker('Area','m^2')
Volume = quantity_maker('Volume','m^3')

Porosity = quantity_maker('Porosity','m^3/m^3')
print(Porosity.units)

Velocity = quantity_maker('Velocity','m/s')
@typechecked
def speed(name:str, l:Length, dt:Time) -> Velocity:
    name = f'{name} = \\frac{{{l.name}}}{{{dt.name}}}'
    return Velocity(name = name, quantity = (l/dt))

Density = quantity_maker('Density','kg/m^3')
@typechecked
def density(name:str, m:Mass, v:Volume) -> Density:
    name = f'{name} = \\frac{{{m.name}}}{{{v.name}}}'
    return Density(name = name, quantity = (m/v))

m = Mass(name='m', magnitude=ufloat(6, 0.1), units='kg')
display(m)

v = Volume(name='v', quantity=Q_(ufloat(3, 0.2),'m^3'))
display(v)

rho_1 = density(r'\zeta',m,v)
display(rho_1)

m2 = Mass(name=r'm_2', magnitude=ufloat(6, 0.1), units='kg')
display(m2)

rho_2 = density(r'\rho_2',m2,v)
display(rho_2)

m^3/m^3


<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [28]:
m0 = Mass(name='m_0', limits=Limits([Q_('2.8 kg'),Q_('6.07 kg')]))
v0 = Volume(name='v_0', limits=Limits([Q_('3.21 m^3'),Q_('3.6 m^3')]))
print(v0*m0)
rho_0 = density(r'\zeta',m0,v0)
display(rho_0)

interval([8.987999999999998, 21.852000000000004])


<IPython.core.display.Latex object>

In [ ]:
Stress = quantity_maker('Stress','kg/m^3')

@dataclass
class Mineral:
    density: Density
    bulk_modulus: Stress
        
@dataclass
class Rock:
    minerals: list[Mineral]    

quartz = Mineral()
calcite = Mineral()
dolomite = Mineral()

dolomite = Rock([1,2,3])
print(dolomite)